In [1]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import pandas_profiling as pp
import missingno as mno
from sklearn import linear_model
from prettytable import PrettyTable
import random
from missingpy import MissForest
from sklearn import tree
from math import floor
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [8]:
bank = pd.read_csv('bank.csv')

In [9]:
bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [10]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [11]:
df = bank[['age','balance','day','duration','campaign','previous']]
df

,age,balance,day,duration,campaign,previous
0,59,2343,5,1042,1,0
1,56,45,5,1467,1,0
2,41,1270,5,1389,1,0
3,55,2476,5,579,1,0
4,54,184,5,673,2,0
...,...,...,...,...,...,...
11157,33,1,20,257,1,0
11158,39,733,16,83,4,0
11159,32,29,19,156,2,0
11160,43,0,8,9,2,5


In [12]:
df.isnull().sum()

age         0
balance     0
day         0
duration    0
campaign    0
previous    0
dtype: int64

In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,11162.0,41.231948,11.913369,18.0,32.0,39.0,49.0,95.0
balance,11162.0,1528.538524,3225.413326,-6847.0,122.0,550.0,1708.0,81204.0
day,11162.0,15.658036,8.420740,1.0,8.0,15.0,22.0,31.0
duration,11162.0,371.993818,347.128386,2.0,138.0,255.0,496.0,3881.0
campaign,11162.0,2.508421,2.722077,1.0,1.0,2.0,3.0,63.0
previous,11162.0,0.832557,2.292007,0.0,0.0,0.0,1.0,58.0


## 20% imputation

In [14]:
bank_orig = df.copy()

In [15]:
data_1 = df.copy()

In [16]:
data_1 = data_1.iloc[:,1:6].mask(np.random.random(data_1.iloc[:,1:6].shape) < .2)
data_1

,balance,day,duration,campaign,previous
0,2343.0,NaN,1042.0,1.0,0.0
1,45.0,5.0,NaN,NaN,0.0
2,1270.0,5.0,1389.0,1.0,NaN
3,2476.0,5.0,579.0,NaN,0.0
4,184.0,5.0,673.0,2.0,0.0
...,...,...,...,...,...
11157,NaN,20.0,257.0,1.0,0.0
11158,733.0,NaN,83.0,4.0,0.0
11159,NaN,NaN,156.0,2.0,0.0
11160,0.0,8.0,9.0,2.0,5.0


In [17]:
data_1['age'] = df['age']
df_one = data_1.copy()

In [18]:
df_one.isnull().sum()

balance     2205
day         2252
duration    2184
campaign    2218
previous    2305
age            0
dtype: int64

### Using KNN imputer

In [19]:
df_knn_c = df_one.copy()
df_knn_c['age'] = df_knn_c['age'].astype('category')

In [20]:
df_knn_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   balance   8957 non-null   float64 
 1   day       8910 non-null   float64 
 2   duration  8978 non-null   float64 
 3   campaign  8944 non-null   float64 
 4   previous  8857 non-null   float64 
 5   age       11162 non-null  category
dtypes: category(1), float64(5)
memory usage: 450.1 KB


In [21]:
df_knn_c.isnull().sum()

balance     2205
day         2252
duration    2184
campaign    2218
previous    2305
age            0
dtype: int64

In [22]:
df_knn = df_knn_c.drop('age',axis=1)
df_knn

,balance,day,duration,campaign,previous
0,2343.0,NaN,1042.0,1.0,0.0
1,45.0,5.0,NaN,NaN,0.0
2,1270.0,5.0,1389.0,1.0,NaN
3,2476.0,5.0,579.0,NaN,0.0
4,184.0,5.0,673.0,2.0,0.0
...,...,...,...,...,...
11157,NaN,20.0,257.0,1.0,0.0
11158,733.0,NaN,83.0,4.0,0.0
11159,NaN,NaN,156.0,2.0,0.0
11160,0.0,8.0,9.0,2.0,5.0


In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_knn = pd.DataFrame(scaler.fit_transform(df_knn), columns = df_knn.columns)

In [24]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)
df_output = pd.DataFrame(imputer.fit_transform(df_knn),columns = df_knn.columns)
df_transform = pd.DataFrame(scaler.inverse_transform(df_output), columns= df_output.columns)

In [26]:
bank_orig

,age,balance,day,duration,campaign,previous
0,59,2343,5,1042,1,0
1,56,45,5,1467,1,0
2,41,1270,5,1389,1,0
3,55,2476,5,579,1,0
4,54,184,5,673,2,0
...,...,...,...,...,...,...
11157,33,1,20,257,1,0
11158,39,733,16,83,4,0
11159,32,29,19,156,2,0
11160,43,0,8,9,2,5


### Now evaluating KNN imputer using absolute error

In [28]:
# Add imputed values as columns to the untouched dataset
bank_orig['knn_balance'] = df_transform['balance']
bank_orig['knn_day'] = df_transform['day']
bank_orig['knn_duration'] = df_transform['duration']
bank_orig['knn_campaign'] = df_transform['campaign']
bank_orig['knn_previous'] = df_transform['previous']
comparison_df = bank_orig[['balance', 'knn_balance', 'day' , 'knn_day' , 'duration' , 'knn_duration' , 'campaign', 'knn_campaign', 'previous', 'knn_previous']]


In [29]:
# Calculate absolute errors
comparison_df['ABS_ERROR_balance'] = round(np.abs(comparison_df['balance'] - comparison_df['knn_balance']),3)
comparison_df['ABS_ERROR_day'] = round(np.abs(comparison_df['day'] - comparison_df['knn_day']),3)
comparison_df['ABS_ERROR_duration'] = round(np.abs(comparison_df['duration'] - comparison_df['knn_duration']),3)
comparison_df['ABS_ERROR_campaign'] = round(np.abs(comparison_df['campaign'] - comparison_df['knn_campaign']),3)
comparison_df['ABS_ERROR_previous'] = round(np.abs(comparison_df['previous'] - comparison_df['knn_previous']),3)

comparison_df.head(50)

,balance,knn_balance,day,knn_day,duration,knn_duration,campaign,knn_campaign,previous,knn_previous,ABS_ERROR_balance,ABS_ERROR_day,ABS_ERROR_duration,ABS_ERROR_campaign,ABS_ERROR_previous
0,2343,2343.000000,5,18.700000,1042,1042.0000,1,1.000000,0,0.000000,0.000,13.700,0.00,0.000,0.000
1,45,45.000000,5,5.000000,1467,700.3000,1,4.000000,0,0.000000,0.000,0.000,766.70,3.000,0.000
2,1270,1270.000000,5,5.000000,1389,1389.0000,1,1.000000,0,0.800000,0.000,0.000,0.00,0.000,0.800
3,2476,2476.000000,5,5.000000,579,579.0000,1,1.900000,0,0.000000,0.000,0.000,0.00,0.900,0.000
4,184,184.000000,5,5.000000,673,673.0000,2,2.000000,0,0.000000,0.000,0.000,0.00,0.000,0.000
5,0,0.000000,5,5.000000,562,562.0000,2,2.000000,0,0.000000,0.000,0.000,0.00,0.000,0.000
6,830,830.000000,6,15.900000,1201,1201.0000,1,1.000000,0,0.000000,0.000,9.900,0.00,0.000,0.000
7,545,545.000000,6,6.000000,1030,1030.0000,1,1.000000,0,0.000000,0.000,0.000,0.00,0.000,0.000
8,1,1.000000,6,6.000000,608,608.0000,1,1.000000,0,0.000000,0.000,0.000,0.00,0.000,0.000
9,5090,5090.000000,6,6.000000,1297,1297.0000,3,4.000000,0,0.000000,0.000,0.000,0.00,1.000,0.000


### Decision Tree imputation

In [30]:
bank_orig = df.copy()
df_dt_c = df_one.copy()
df_dt_c['age'] = df_dt_c['age'].astype('category')

In [31]:
df_dt_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   balance   8957 non-null   float64 
 1   day       8910 non-null   float64 
 2   duration  8978 non-null   float64 
 3   campaign  8944 non-null   float64 
 4   previous  8857 non-null   float64 
 5   age       11162 non-null  category
dtypes: category(1), float64(5)
memory usage: 450.1 KB


In [32]:
df_dt_c.isnull().sum()

balance     2205
day         2252
duration    2184
campaign    2218
previous    2305
age            0
dtype: int64

In [33]:
df_dt = df_dt_c
df_dt

,balance,day,duration,campaign,previous,age
0,2343.0,NaN,1042.0,1.0,0.0,59
1,45.0,5.0,NaN,NaN,0.0,56
2,1270.0,5.0,1389.0,1.0,NaN,41
3,2476.0,5.0,579.0,NaN,0.0,55
4,184.0,5.0,673.0,2.0,0.0,54
...,...,...,...,...,...,...
11157,NaN,20.0,257.0,1.0,0.0,33
11158,733.0,NaN,83.0,4.0,0.0,39
11159,NaN,NaN,156.0,2.0,0.0,32
11160,0.0,8.0,9.0,2.0,5.0,43


In [34]:
#Standardize numerical Values
def standardize_col(col):
    return (col - col.min()) * 1.0 / (col.max() - col.min())

In [35]:
#order the columns of the dataframe according to decreasing number of null values, return list of column names
def ordered_cols(df):
    x = df.isna().sum()
    x = x.sort_values()
    x = x.to_frame()
    columns_sorted = x.index.values
    return list(columns_sorted)

In [36]:
class DecisionTree(object):
    """
    Class to create decision tree model (CART)
    """
    def __init__(self, _max_depth, _min_splits):
        self.max_depth = _max_depth
        self.min_splits = _min_splits

    def fit(self, _feature, _label):
        """
        :param _feature:
        :param _label:
        :return:
        """
        self.feature = _feature
        self.label = _label
        self.train_data = np.column_stack((self.feature,self.label))
        self.build_tree()


    def compute_gini_similarity(self, groups, class_labels):
        """
        compute the gini index for the groups and class labels
        :param groups:
        :param class_labels:
        :return:
        """
        num_sample = sum([len(group) for group in groups])
        gini_score = 0

        for group in groups:
            size = float(len(group))

            if size == 0:
                continue
            score = 0.0
            for label in class_labels:
                porportion = (group[:,-1] == label).sum() / size
                score += porportion * porportion
            gini_score += (1.0 - score) * (size/num_sample)

        return gini_score

    def terminal_node(self, _group):
        """
        Function set terminal node as the most common class in the group to make prediction later on
        is an helper function used to mark the leaf node in the tree based on the early stop condition
        or actual stop condition which ever is meet early
        :param _group:
        :return:
        """
        class_labels, count = np.unique(_group[:,-1], return_counts= True)
        return class_labels[np.argmax(count)]

    def split(self, index, val, data):
        """
        split features into two groups based on their values
        :param index:
        :param val:
        :param data:
        :return:
        """
        data_left = np.array([]).reshape(0,self.train_data.shape[1])
        data_right = np.array([]).reshape(0, self.train_data.shape[1])

        for row in data:
            if row[index] <= val :
                data_left = np.vstack((data_left,row))

            if row[index] > val:
                data_right = np.vstack((data_right, row))

        return data_left, data_right

    def best_split(self, data):
        """
        find the best split information using the gini score
        :param data:
        :return best_split result dict:
        """
        class_labels = np.unique(data[:,-1])
        best_index = 999
        best_val = 999
        best_score = 999
        best_groups = None

        for idx in range(data.shape[1]-1):
            for row in data:
                groups = self.split(idx, row[idx], data)
                gini_score = self.compute_gini_similarity(groups,class_labels)

                if gini_score < best_score:
                    best_index = idx
                    best_val = row[idx]
                    best_score = gini_score
                    best_groups = groups
        result = {}
        result['index'] = best_index
        result['val'] = best_val
        result['groups'] = best_groups
        return result


    def split_branch(self, node, depth):
        """
        recursively split the data and
        check for early stop argument based on self.max_depth and self.min_splits
        - check if left or right groups are empty is yess craete terminal node
        - check if we have reached max_depth early stop condition if yes create terminal node
        - Consider left node, check if the group is too small using min_split condition
            - if yes create terminal node
            - else continue to build the tree
        - same is done to the right side as well.
        else
        :param node:
        :param depth:
        :return:
        """
        left_node , right_node = node['groups']
        del(node['groups'])

        if not isinstance(left_node,np.ndarray) or not isinstance(right_node,np.ndarray):
            node['left'] = self.terminal_node(left_node + right_node)
            node['right'] = self.terminal_node(left_node + right_node)
            return

        if depth >= self.max_depth:
            node['left'] = self.terminal_node(left_node)
            node['right'] = self.terminal_node(right_node)
            return

        if len(left_node) <= self.min_splits:
            node['left'] = self.terminal_node(left_node)
        else:
            node['left'] = self.best_split(left_node)
            self.split_branch(node['left'],depth + 1)


        if len(right_node) <= self.min_splits:
            node['right'] = self.terminal_node(right_node)
        else:
            node['right'] = self.best_split(right_node)
            self.split_branch(node['right'],depth + 1)

    def build_tree(self):
        """
        build tree recursively with help of split_branch function
         - Create a root node
         - call recursive split_branch to build the complete tree
        :return:
        """
        self.root = self.best_split(self.train_data)
        self.split_branch(self.root, 1)
        return self.root

    def _predict(self, node, row):
        """
        Recursively traverse through the tress to determine the
        class of unseen sample data point during prediction
        :param node:
        :param row:
        :return:
        """
        if row[node['index']] < node['val']:
            if isinstance(node['left'], dict):
                return self._predict(node['left'], row)
            else:
                return node['left']

        else:
            if isinstance(node['right'],dict):
                return self._predict(node['right'],row)
            else:
                return node['right']

    def predict(self, test_data):
        """
        predict the set of data point
        :param test_data:
        :return:
        """
        self.predicted_label = np.array([])
        for idx in test_data:
            self.predicted_label = np.append(self.predicted_label, self._predict(self.root,idx))

        return self.predicted_label

In [37]:
def impute(df):
  #initial imputed values
  
  Z = np.matrix(df['age']).reshape(-1,1)
  indexes_dict = {}
  
  #Get ordered list of col names except the session id which is always filled
  col_partial = ordered_cols(df.loc[:, df.columns != 'age'])
  col_list = col_partial
#   print(col_partial)
  
  
  for col in col_partial:
  #   print(col)
    y  = df[col]
  #   print(type(y))
    pos = list(y[y.isnull()].index)
  #   print(pos)
    indexes_dict[col] = pos

    X_train = np.delete(Z, pos, axis=0)
    X_test = Z[pos, :]
    y_train = np.matrix(y.drop(y.index[pos])).reshape(Z.shape[0]-len(pos),1)

    y_final = np.matrix(y).reshape(Z.shape[0],1)

  #   print(y.values.reshape(Z.shape[0],1)[pos[0]])
  #   X_test = Z.loc[pos].values#.reshape(1,-1)
  #   y_train = (y.drop(y.index[pos])).values(columns = 1)
  #   X_train = (Z.drop(Z.index[pos])).values
  #   X_train = (X_train).values#.reshape(-1,1)

    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(X_train, y_train)

    predicted = clf.predict(X_test)
  #   print(predicted)

    for i in range(len(pos)):
      ind = pos[i]
      y_final[ind] = predicted[i]

    Z = np.concatenate((Z,y_final),axis = 1)
  #   break
  
  #Converge 10 times
  for l in range(10):
    for colm in range(1,Z.shape[1]):
      column_name = col_partial[colm-1]
      pos = indexes_dict[column_name]

      X_train = np.delete(Z, colm, axis=1)
      X_train = np.delete(X_train, pos, axis=0)
      X_test = np.delete(Z[pos, :], colm, axis=1)
      y_train = Z[:, colm]
      y_train = np.delete(y_train, pos, axis=0)

  #     print(X_train.shape)
  #     print(X_test.shape)
  #     print(y_train.shape)

      clf = tree.DecisionTreeRegressor()
      clf = clf.fit(X_train, y_train)

      predicted = clf.predict(X_test)

      for i in range(len(pos)):
        ind = pos[i]
        Z[ind, colm] = predicted[i]
  
  col_list =  ['age'] + col_list
#   print(col_list)
  return pd.DataFrame(Z, columns=col_list)

In [38]:
df_transform= impute(df_dt)
df_transform.isnull().sum()

/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.w

/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.w

/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.w

/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.w

age         0
duration    0
balance     0
campaign    0
day         0
previous    0
dtype: int64

In [44]:
df_transform

,output,cp,fbs,sex,slp,exng,thall,restecg,thalachh,oldpeak,caa,age,trtbps,chol
0,1.0,3.0,0.0,1.0,0.0,0.0,2.0,0.0,150.0,2.3,0.0,63.0,145.0,233.0
1,1.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,187.0,3.5,0.0,43.0,130.0,315.0
2,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,171.0,1.4,0.0,37.0,130.0,204.0
3,1.0,2.0,0.0,0.0,2.0,0.0,2.0,1.0,158.0,0.8,0.0,56.0,132.0,236.0
4,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,163.0,0.0,0.0,42.0,120.0,354.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,123.0,2.0,0.0,57.0,140.0,241.0
299,0.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,132.0,1.2,1.0,45.0,110.0,264.0
300,0.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,141.0,1.5,2.0,68.0,144.0,193.0
301,0.0,0.0,0.0,1.0,1.0,1.0,3.0,0.0,114.0,1.2,1.0,57.0,112.0,131.0


In [39]:
bank_orig

,age,balance,day,duration,campaign,previous
0,59,2343,5,1042,1,0
1,56,45,5,1467,1,0
2,41,1270,5,1389,1,0
3,55,2476,5,579,1,0
4,54,184,5,673,2,0
...,...,...,...,...,...,...
11157,33,1,20,257,1,0
11158,39,733,16,83,4,0
11159,32,29,19,156,2,0
11160,43,0,8,9,2,5


In [40]:
# Add imputed values as columns to the untouched dataset
bank_orig['dt_balance'] = df_transform['balance']
bank_orig['dt_day'] = df_transform['day']
bank_orig['dt_duration'] = df_transform['duration']
bank_orig['dt_campaign'] = df_transform['campaign']
bank_orig['dt_previous'] = df_transform['previous']
comparison_df = bank_orig[['balance', 'dt_balance', 'day' , 'dt_day' , 'duration' , 'dt_duration' , 'campaign', 'dt_campaign', 'previous', 'dt_previous']]


In [42]:
# Calculate absolute errors
comparison_df['ABS_ERROR_balance'] = round(np.abs(comparison_df['balance'] - comparison_df['dt_balance']),3)
comparison_df['ABS_ERROR_day'] = round(np.abs(comparison_df['day'] - comparison_df['dt_day']),3)
comparison_df['ABS_ERROR_duration'] = round(np.abs(comparison_df['duration'] - comparison_df['dt_duration']),3)
comparison_df['ABS_ERROR_campaign'] = round(np.abs(comparison_df['campaign'] - comparison_df['dt_campaign']),3)
comparison_df['ABS_ERROR_previous'] = round(np.abs(comparison_df['previous'] - comparison_df['dt_previous']),3)

comparison_df.head(50)

,balance,dt_balance,day,dt_day,duration,dt_duration,campaign,dt_campaign,previous,dt_previous,ABS_ERROR_balance,ABS_ERROR_day,ABS_ERROR_duration,ABS_ERROR_campaign,ABS_ERROR_previous
0,2343,2343.0,5,23.0,1042,1042.0,1,1.0,0,0.0,0.0,18.0,0.0,0.0,0.0
1,45,45.0,5,5.0,1467,157.0,1,2.0,0,0.0,0.0,0.0,1310.0,1.0,0.0
2,1270,1270.0,5,5.0,1389,1389.0,1,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,2476,2476.0,5,5.0,579,579.0,1,2.0,0,0.0,0.0,0.0,0.0,1.0,0.0
4,184,184.0,5,5.0,673,673.0,2,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,0.0,5,5.0,562,562.0,2,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0
6,830,830.0,6,16.0,1201,1201.0,1,1.0,0,0.0,0.0,10.0,0.0,0.0,0.0
7,545,545.0,6,6.0,1030,1030.0,1,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,1.0,6,6.0,608,608.0,1,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0
9,5090,5090.0,6,6.0,1297,1297.0,3,2.0,0,0.0,0.0,0.0,0.0,1.0,0.0


### Using Missforest imputation

In [43]:
bank_orig = df.copy()
df_mf = df_one.copy()
df_mf['age'] = df_mf['age'].astype('category')

In [44]:
df_mf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   balance   8957 non-null   float64 
 1   day       8910 non-null   float64 
 2   duration  8978 non-null   float64 
 3   campaign  8944 non-null   float64 
 4   previous  8857 non-null   float64 
 5   age       11162 non-null  category
dtypes: category(1), float64(5)
memory usage: 450.1 KB


In [45]:
df_mf.isnull().sum()

balance     2205
day         2252
duration    2184
campaign    2218
previous    2305
age            0
dtype: int64

In [46]:
imputer = MissForest()
df_transform = pd.DataFrame(imputer.fit_transform(df_mf.drop('age',axis=1)),columns = df_mf.drop('age',axis=1).columns)
df_transform

/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.

Iteration: 0


/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.

Iteration: 1


/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.

Iteration: 2


/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:387: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/ab0707/opt/anaconda3/lib/python3.

Iteration: 3


,balance,day,duration,campaign,previous
0,2343.000000,18.05,1042.0,1.00,0.00
1,45.000000,5.00,473.6,1.76,0.00
2,1270.000000,5.00,1389.0,1.00,0.58
3,2476.000000,5.00,579.0,1.74,0.00
4,184.000000,5.00,673.0,2.00,0.00
...,...,...,...,...,...
11157,2523.153119,20.00,257.0,1.00,0.00
11158,733.000000,20.03,83.0,4.00,0.00
11159,2837.820000,14.40,156.0,2.00,0.00
11160,0.000000,8.00,9.0,2.00,5.00


In [47]:
df_transform

,balance,day,duration,campaign,previous
0,2343.000000,18.05,1042.0,1.00,0.00
1,45.000000,5.00,473.6,1.76,0.00
2,1270.000000,5.00,1389.0,1.00,0.58
3,2476.000000,5.00,579.0,1.74,0.00
4,184.000000,5.00,673.0,2.00,0.00
...,...,...,...,...,...
11157,2523.153119,20.00,257.0,1.00,0.00
11158,733.000000,20.03,83.0,4.00,0.00
11159,2837.820000,14.40,156.0,2.00,0.00
11160,0.000000,8.00,9.0,2.00,5.00


In [49]:
bank_orig

,age,balance,day,duration,campaign,previous
0,59,2343,5,1042,1,0
1,56,45,5,1467,1,0
2,41,1270,5,1389,1,0
3,55,2476,5,579,1,0
4,54,184,5,673,2,0
...,...,...,...,...,...,...
11157,33,1,20,257,1,0
11158,39,733,16,83,4,0
11159,32,29,19,156,2,0
11160,43,0,8,9,2,5


### Now evaluating MissForest Imputer using absolute error

In [50]:
# Add imputed values as columns to the untouched dataset
bank_orig['mf_balance'] = df_transform['balance']
bank_orig['mf_day'] = df_transform['day']
bank_orig['mf_duration'] = df_transform['duration']
bank_orig['mf_campaign'] = df_transform['campaign']
bank_orig['mf_previous'] = df_transform['previous']
comparison_df = bank_orig[['balance', 'mf_balance', 'day' , 'mf_day' , 'duration' , 'mf_duration' , 'campaign', 'mf_campaign', 'previous', 'mf_previous']]


In [51]:
# Calculate absolute errors
comparison_df['ABS_ERROR_balance'] = round(np.abs(comparison_df['balance'] - comparison_df['mf_balance']),3)
comparison_df['ABS_ERROR_day'] = round(np.abs(comparison_df['day'] - comparison_df['mf_day']),3)
comparison_df['ABS_ERROR_duration'] = round(np.abs(comparison_df['duration'] - comparison_df['mf_duration']),3)
comparison_df['ABS_ERROR_campaign'] = round(np.abs(comparison_df['campaign'] - comparison_df['mf_campaign']),3)
comparison_df['ABS_ERROR_previous'] = round(np.abs(comparison_df['previous'] - comparison_df['mf_previous']),3)

comparison_df.head(50)

,balance,mf_balance,day,mf_day,duration,mf_duration,campaign,mf_campaign,previous,mf_previous,ABS_ERROR_balance,ABS_ERROR_day,ABS_ERROR_duration,ABS_ERROR_campaign,ABS_ERROR_previous
0,2343,2343.000,5,18.05,1042,1042.000000,1,1.00,0,0.00,0.000,13.05,0.000,0.00,0.00
1,45,45.000,5,5.00,1467,473.600000,1,1.76,0,0.00,0.000,0.00,993.400,0.76,0.00
2,1270,1270.000,5,5.00,1389,1389.000000,1,1.00,0,0.58,0.000,0.00,0.000,0.00,0.58
3,2476,2476.000,5,5.00,579,579.000000,1,1.74,0,0.00,0.000,0.00,0.000,0.74,0.00
4,184,184.000,5,5.00,673,673.000000,2,2.00,0,0.00,0.000,0.00,0.000,0.00,0.00
5,0,0.000,5,5.00,562,562.000000,2,2.00,0,0.00,0.000,0.00,0.000,0.00,0.00
6,830,830.000,6,18.33,1201,1201.000000,1,1.00,0,0.00,0.000,12.33,0.000,0.00,0.00
7,545,545.000,6,6.00,1030,1030.000000,1,1.00,0,0.00,0.000,0.00,0.000,0.00,0.00
8,1,1.000,6,6.00,608,608.000000,1,1.00,0,0.00,0.000,0.00,0.000,0.00,0.00
9,5090,5090.000,6,6.00,1297,1297.000000,3,4.61,0,0.00,0.000,0.00,0.000,1.61,0.00
